### [예제 3] 은닉층 1개를 가지는 LogicGate class 

In [1]:
import numpy as np
from datetime import datetime

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = float(tmp_val) - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [2]:
# LogicGate Class

class LogicGate:
    
    # 생성자
    # xdata, tdata => numpy.array(...)
    def __init__(self, gate_name, xdata, tdata, input_nodes, hidden_1_nodes, output_nodes, learning_rate, iteration_count):
        
        self.name = gate_name
        
        # 입력 데이터, 정답 데이터 초기화
        self.xdata = xdata.reshape(4, 2)
        self.tdata = tdata.reshape(4, 1)
        
        # 2층 hidden layer unit : 
        # 가중치 W, 바이어스 b 초기화
        self.W2 = np.random.rand(input_nodes, hidden_1_nodes)  
        self.b2 = np.random.rand(hidden_1_nodes)
        
        # 3층 output layer unit : 1 개 
        self.W3 = np.random.rand(hidden_1_nodes, output_nodes)
        self.b3 = np.random.rand(output_nodes)
                        
        # 학습률 learning rate 초기화
        self.learning_rate = learning_rate
        
        # 반복횟수 초기화
        self.iteration_count = iteration_count
    
        print(self.name + " object is created")
        
    # 손실함수
    def feed_forward(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.xdata, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )
    
    # obtain W and b
    def get_W_b(self):
        
        return self.W2,  self.b2, self.W3, self.b3
    
    # 손실 값 계산
    def loss_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.xdata, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )
    
    # query, 즉 미래 값 예측 함수
    def predict(self, xdata):
        
        z2 = np.dot(xdata, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        if y >= 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result

    # accuracy method 
    def accuracy(self, test_data):
        
        matched_list = []
        not_matched_list = []
        
        input_data = test_data[ :, 0:-1]
        target_data = test_data[ :, -1]
        
        for index in range(len(input_data)):
            
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_val = len(matched_list) / len(input_data)
        
        return accuracy_val
        
        
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self):
        
        f = lambda x : self.feed_forward()
        
        print("Initial loss value = ", self.loss_val())
        
        start_time = datetime.now()
        
        for step in  range(self.iteration_count):
            
            self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
    
            self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        
            self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
    
            self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)
    
            if (step % 500 == 0):
                print("step = ", step, "loss value = ", self.loss_val())
                
        end_time = datetime.now()
        
        print("")
        print("Elapsed Time => ", end_time - start_time)

In [3]:
and_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
and_tdata = np.array([0, 0, 0, 1])

input_nodes = 2
hidden_1_nodes = 6
output_nodes = 1

lr = 1e-1  # learning rate
count = 10001  # iteration count

# AND Gate 객체 생성 및 training
and_obj = LogicGate("AND", and_xdata, and_tdata, input_nodes, hidden_1_nodes, output_nodes, lr, count)

and_obj.train()

AND object is created
Initial loss value =  9.688315119949811
step =  0 loss value =  6.57241771839348
step =  500 loss value =  0.20500012626685357
step =  1000 loss value =  0.05682415247631471
step =  1500 loss value =  0.030547673274890136
step =  2000 loss value =  0.020389331758867098
step =  2500 loss value =  0.015126861126972964
step =  3000 loss value =  0.01194505765684446
step =  3500 loss value =  0.009827874321734998
step =  4000 loss value =  0.008324119322008088
step =  4500 loss value =  0.007204324397520117
step =  5000 loss value =  0.006340001595179216
step =  5500 loss value =  0.005653836987183084
step =  6000 loss value =  0.005096631910769077
step =  6500 loss value =  0.004635644750438157
step =  7000 loss value =  0.004248271912738093
step =  7500 loss value =  0.003918423773687928
step =  8000 loss value =  0.0036343466455406306
step =  8500 loss value =  0.00338726112303469
step =  9000 loss value =  0.003170481615747668
step =  9500 loss value =  0.00297883

In [6]:
# AND Gate prediction
print(and_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 1] ])

accuracy_ret = and_obj.accuracy(test_data)

print(and_obj.name, ' Accuracy => ', accuracy_ret)

AND 

AND  Accuracy =>  1.0


In [7]:
or_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
or_tdata = np.array([0, 1, 1, 1])

input_nodes = 2
hidden_1_nodes = 6
output_nodes = 1

lr = 1e-1  # learning rate
count = 10001  # iteration count

# OR Gate 객체 생성 및 training
or_obj = LogicGate("OR", or_xdata, or_tdata, input_nodes, hidden_1_nodes, output_nodes, lr, count)

or_obj.train()

OR object is created
Initial loss value =  2.582609979895192
step =  0 loss value =  2.435022872149788
step =  500 loss value =  0.12542441032934573
step =  1000 loss value =  0.039404260489446695
step =  1500 loss value =  0.02171546451924782
step =  2000 loss value =  0.014600008932195806
step =  2500 loss value =  0.010854943293768592
step =  3000 loss value =  0.00857361325340717
step =  3500 loss value =  0.007049911576431077
step =  4000 loss value =  0.005965674121673579
step =  4500 loss value =  0.005157623192055102
step =  5000 loss value =  0.004533794988020166
step =  5500 loss value =  0.004038633886678442
step =  6000 loss value =  0.003636691324563523
step =  6500 loss value =  0.00330433234197611
step =  7000 loss value =  0.003025219415229454
step =  7500 loss value =  0.0027877132796615624
step =  8000 loss value =  0.0025833066537750415
step =  8500 loss value =  0.002405643020359796
step =  9000 loss value =  0.0022498811414817656
step =  9500 loss value =  0.002112

In [8]:
# OR Gate prediction
print(or_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1] ])

accuracy_ret = or_obj.accuracy(test_data)

print(or_obj.name, ' Accuracy => ', accuracy_ret)

OR 

OR  Accuracy =>  1.0


In [9]:
nand_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
nand_tdata = np.array([1, 1, 1, 0])

input_nodes = 2
hidden_1_nodes = 6
output_nodes = 1

lr = 1e-1  # learning rate
count = 10001  # iteration count

# NAND Gate 객체 생성 및 training
nand_obj = LogicGate("NAND", nand_xdata, nand_tdata, input_nodes, hidden_1_nodes, output_nodes, lr, count)

nand_obj.train()

NAND object is created
Initial loss value =  3.537699315382069
step =  0 loss value =  3.2325806674417894
step =  500 loss value =  0.1796290200872674
step =  1000 loss value =  0.052937040855950704
step =  1500 loss value =  0.02874896799442299
step =  2000 loss value =  0.01922157310959876
step =  2500 loss value =  0.014250881947258268
step =  3000 loss value =  0.01123646734446544
step =  3500 loss value =  0.009228302091079388
step =  4000 loss value =  0.0078016209326189105
step =  4500 loss value =  0.006739495977002475
step =  5000 loss value =  0.005920140344601117
step =  5500 loss value =  0.005270147641777478
step =  6000 loss value =  0.004742754996469717
step =  6500 loss value =  0.004306821015377259
step =  7000 loss value =  0.003940838812280453
step =  7500 loss value =  0.0036294969587374196
step =  8000 loss value =  0.0033616102027700613
step =  8500 loss value =  0.0031288246731455962
step =  9000 loss value =  0.002924780105566975
step =  9500 loss value =  0.002

In [10]:
# NAND Gate prediction
print(nand_obj.name, "\n")

test_data = np.array([ [0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

accuracy_ret = nand_obj.accuracy(test_data)

print(nand_obj.name, ' Accuracy => ', accuracy_ret)

NAND 

NAND  Accuracy =>  1.0


In [11]:
xor_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
xor_tdata = np.array([0, 1, 1, 0])

input_nodes = 2
hidden_1_nodes = 6
output_nodes = 1

lr = 1e-1  # learning rate
count = 10001  # iteration count

# XOR Gate 객체 생성 및 training
xor_obj = LogicGate("XOR", xor_xdata, xor_tdata, input_nodes, hidden_1_nodes, output_nodes, lr, count)

xor_obj.train()

XOR object is created
Initial loss value =  4.6476530237819595
step =  0 loss value =  3.743419902075723
step =  500 loss value =  2.5202215644198556
step =  1000 loss value =  1.0892732837440944
step =  1500 loss value =  0.2298413041591701
step =  2000 loss value =  0.10375128382046522
step =  2500 loss value =  0.06365954694037695
step =  3000 loss value =  0.04499251199327672
step =  3500 loss value =  0.03443065949433042
step =  4000 loss value =  0.027713266739881705
step =  4500 loss value =  0.023095850770986567
step =  5000 loss value =  0.0197416886957727
step =  5500 loss value =  0.01720261975189127
step =  6000 loss value =  0.01521822325904175
step =  6500 loss value =  0.013627384402912718
step =  7000 loss value =  0.012325344977275138
step =  7500 loss value =  0.011241181257944719
step =  8000 loss value =  0.010325240520265535
step =  8500 loss value =  0.009541766669373451
step =  9000 loss value =  0.008864382902407236
step =  9500 loss value =  0.00827322212659632

In [12]:
# XOR Gate prediction
print(xor_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

accuracy_ret = xor_obj.accuracy(test_data)

print(xor_obj.name, ' Accuracy => ', accuracy_ret)

XOR 

XOR  Accuracy =>  1.0
